  <h1><center><b> Language Identification </b></center></h1> 
  <h2><center><b> Presented By: </b></center></h2> 
  

  <hr>

<img src="https://miro.medium.com/max/2402/1*1AtFVKl2VEeQHzwoEtbdow.png" width="400px" height="250px" align="right">

<h2> <b> 04 Aishwarya Athawale </h2>
<h2><b> 08 Pankti Fadia </b></h2>
<h2><b> 10 Hetvi Julasana </b></h2>








In [ ]:
import random
from collections import Counter
import numpy as np

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Embedding, Conv1D, MaxPool1D, Flatten, Dense
from keras.models import Model

In [ ]:
# Library versions
print(f'keras= {keras.__version__}')
print(f'sklearn= {sklearn.__version__}')
print(f'numpy= {np.__version__}')

keras= 2.4.3
sklearn= 0.22.2.post1
numpy= 1.18.5


# **Initialize basic configurations**

In [ ]:
# Basic Configs
data_dir = '/content/drive/My Drive/Colab Notebooks/data_dir'
# Select to articles from file
num_of_articles = 10000
# Maximum sequence length
sentense_len = 150
# shingle configs
shingles_range = (70, 100, 130)
# how many shingles generate per line
shingle_per_line = 10 
# out of vocabulary token
oov_str = 'oov'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# language code wise full name mapping
lang_code_dict = {
    'en':'english', 'de':'german', 
    'fr':'french', 'it':'italian', 
    'es':'spanish'
}

# language code wise data file mapping
data_info = {
    'en' : data_dir + '/wiki.en.text',
    'de' : data_dir + '/wiki.de.text',
    'fr' : data_dir + '/wiki.fr.text',
    'it' : data_dir + '/wiki.it.text',
    'es' : data_dir + '/wiki.es.text',
}

# **Load Data**

In [ ]:
for lang_code, file_path in data_info.items():
    print(lang_code, lang_code_dict[lang_code], file_path)

en english /content/drive/My Drive/Colab Notebooks/data_dir/wiki.en.text
de german /content/drive/My Drive/Colab Notebooks/data_dir/wiki.de.text
fr french /content/drive/My Drive/Colab Notebooks/data_dir/wiki.fr.text
it italian /content/drive/My Drive/Colab Notebooks/data_dir/wiki.it.text
es spanish /content/drive/My Drive/Colab Notebooks/data_dir/wiki.es.text


In [ ]:
# data loading
data_dict = {}
for lang_code, file_path in data_info.items():
    with open(file_path, encoding='ISO-8859-1') as file:
        lines = file.readlines()
        lines = lines[:num_of_articles]
        # convert to lower case
        lines = [l.lower().strip() for l in lines]
        data_dict[lang_code] = lines
        print(lang_code, len(lines))

en 70
de 77
fr 42
it 53
es 40


# **Preprocessing**

In [ ]:
def generate_shingles(line, length, total):
    """
    Generate shingles from line
    """
    
    shingle_list = [] 
    max_index = len(line) - length
    if max_index > 0:
        for _ in range(total):
            index = random.randint(0, max_index)
            shingle_text = line[index:index+length]
            shingle_list.append(shingle_text)
    else:
        shingle_list.append(line)
    return shingle_list

In [ ]:
def generate_shingles_lines(line, length, total):
    """
    Generate shingles from list of lines
    """
    shingle_list = []
    for line in lines:
        shingles = generate_shingles(line=line, length=length, total=total)
        shingle_list.extend(shingles)
    return shingle_list

In [ ]:
# generate shingles
shingle_data_dict = {}
for lang, lines in data_dict.items():
    shingle_list = []
    for s_range in shingles_range:
        shingles = generate_shingles_lines(lines, s_range, shingle_per_line)
        shingle_list.extend(shingles)
    shingle_data_dict[lang] = shingle_list
    print(lang, len(shingle_list))

en 2073
de 2256
fr 1260
it 1590
es 1200


In [ ]:
# create list of lines and labels
data_lines, labels = [], []
for lang, samples in shingle_data_dict.items():
    data_lines.extend(samples)
    total_samples = len(samples)
    labels.extend([lang] * total_samples)
print(len(data_lines), len(labels))

8379 8379


# **Voacabulary Preparation**

In [ ]:
# create list of all characters from all data lines
data_char_ls = []
for line in data_lines:
    char_ls = [c for c in line]
    data_char_ls.append(char_ls)
    
# count all characters
cunt = Counter(x for xs in data_char_ls for x in set(xs))

# create vocabulary
char_vocab = [c[0] for c in cunt.most_common(76)] + [oov_str]
print(char_vocab)

[' ', 'e', 'a', 'i', 'r', 't', 'n', 's', 'l', 'd', 'o', 'c', 'u', 'm', 'p', 'g', 'h', 'b', 'f', 'v', 'w', 'y', 'z', 'k', 'q', 'j', 'é', 'x', 'ü', 'ó', 'í', '?', 'è', 'ö', 'ä', 'á', 'à', 'ñ', '1', 'ò', '0', 'ß', 'ç', 'ù', '9', '5', 'â', '4', 'ú', 'ê', '2', '8', '7', 'ã', 'î', '3', '6', 'ô', 'ì', '\x80', '\x93', '³', '\x9c', '¼', '¤', 'ï', 'û', '\x9f', '¿', '¶', '°', '²', '¸', '\x9a', '©', 'å', 'oov']


In [ ]:
# create dictionary for (char to index)
# here we (index + 1) becoz, 0 index for padding
ch2int = {c:i+1 for i, c in enumerate(char_vocab)}
print(ch2int)
print()
# create dictionary for (index to char)
int2ch = {i:c for c, i in ch2int.items()}
print(int2ch)

{' ': 1, 'e': 2, 'a': 3, 'i': 4, 'r': 5, 't': 6, 'n': 7, 's': 8, 'l': 9, 'd': 10, 'o': 11, 'c': 12, 'u': 13, 'm': 14, 'p': 15, 'g': 16, 'h': 17, 'b': 18, 'f': 19, 'v': 20, 'w': 21, 'y': 22, 'z': 23, 'k': 24, 'q': 25, 'j': 26, 'é': 27, 'x': 28, 'ü': 29, 'ó': 30, 'í': 31, '?': 32, 'è': 33, 'ö': 34, 'ä': 35, 'á': 36, 'à': 37, 'ñ': 38, '1': 39, 'ò': 40, '0': 41, 'ß': 42, 'ç': 43, 'ù': 44, '9': 45, '5': 46, 'â': 47, '4': 48, 'ú': 49, 'ê': 50, '2': 51, '8': 52, '7': 53, 'ã': 54, 'î': 55, '3': 56, '6': 57, 'ô': 58, 'ì': 59, '\x80': 60, '\x93': 61, '³': 62, '\x9c': 63, '¼': 64, '¤': 65, 'ï': 66, 'û': 67, '\x9f': 68, '¿': 69, '¶': 70, '°': 71, '²': 72, '¸': 73, '\x9a': 74, '©': 75, 'å': 76, 'oov': 77}

{1: ' ', 2: 'e', 3: 'a', 4: 'i', 5: 'r', 6: 't', 7: 'n', 8: 's', 9: 'l', 10: 'd', 11: 'o', 12: 'c', 13: 'u', 14: 'm', 15: 'p', 16: 'g', 17: 'h', 18: 'b', 19: 'f', 20: 'v', 21: 'w', 22: 'y', 23: 'z', 24: 'k', 25: 'q', 26: 'j', 27: 'é', 28: 'x', 29: 'ü', 30: 'ó', 31: 'í', 32: '?', 33: 'è', 34: 'ö',

In [ ]:
def encode(in_ls, key):
    """
    encode list of character to index of characters using 'char2int' dictionary
    """
    out_ls = []
    for ch in in_ls:
        index = key.get(ch)
        if index is None:
            index = key.get(oov_str)
        out_ls.append(index)
    return out_ls

In [ ]:
# data encoding
encoded_ls = [encode(l, ch2int) for l in data_lines]
print(len(encoded_ls))

8379


In [ ]:
# observe count 'oov' in dataset
counts = 0
for enc in encoded_ls:
    if ch2int[oov_str] in enc:
        counts += 1
counts

33

In [ ]:
# padding and trucating of encoded sequence
X = pad_sequences(encoded_ls, maxlen=sentense_len, truncating='post', padding='post')

In [ ]:
# target encoding from 'en' or 'de' language code to 0, 1 
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
print(label_encoder.classes_)

['de' 'en' 'es' 'fr' 'it']


In [ ]:
# one hot encoding of targets
y = to_categorical(encoded_labels)

In [ ]:
print(X.shape, y.shape)

(8379, 150) (8379, 5)


In [ ]:
# Train & Test split (70:30) ratio from full data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5865, 150) (2514, 150) (5865, 5) (2514, 5)


# **Model Implementation**

In [ ]:
# Build the Neural network
inp = Input(shape=(sentense_len, ))
#accepts multiple input
x = Embedding(input_dim=len(char_vocab) + 1, output_dim=64)(inp)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPool1D(5)(x)
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPool1D(20)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(5, activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 64)           4992      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 64)           20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 29, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 25, 64)            20544     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=256, epochs=5)

Epoch 1/5
23/23 [==============================] - 4s 184ms/step - loss: 1.5525 - accuracy: 0.3115 - val_loss: 1.4919 - val_accuracy: 0.3846
Epoch 2/5
23/23 [==============================] - 4s 175ms/step - loss: 1.2744 - accuracy: 0.4842 - val_loss: 0.9634 - val_accuracy: 0.6384
Epoch 3/5
23/23 [==============================] - 4s 172ms/step - loss: 0.6933 - accuracy: 0.7558 - val_loss: 0.4789 - val_accuracy: 0.8568
Epoch 4/5
23/23 [==============================] - 4s 181ms/step - loss: 0.3553 - accuracy: 0.8950 - val_loss: 0.3151 - val_accuracy: 0.9021
Epoch 5/5
23/23 [==============================] - 4s 177ms/step - loss: 0.2182 - accuracy: 0.9434 - val_loss: 0.2051 - val_accuracy: 0.9487


# **Evaluation**

In [ ]:
# prediction on test data
pred = model.predict(X_test)

In [ ]:
pred_y = pred.argmax(axis=1).ravel()++z
actual_y = y_test.argmax(axis=1).ravel()

In [ ]:
# Generate classification report
report = classification_report(actual_y, pred_y, target_names=label_encoder.classes_)
print(report)

              precision    recall  f1-score   support

          de       0.97      0.97      0.97       646
          en       0.96      0.96      0.96       645
          es       0.96      0.92      0.94       375
          fr       0.86      0.94      0.90       372
          it       0.98      0.94      0.96       476

    accuracy                           0.95      2514
   macro avg       0.94      0.94      0.94      2514
weighted avg       0.95      0.95      0.95      2514



In [ ]:
def predict(line):
    """
    Prediction method for single line
    """
    line = line.lower()
    chars = [c for c in line]
    encoded = encode(chars, ch2int)
    padded = keras.preprocessing.sequence.pad_sequences([encoded], maxlen=sentense_len, truncating='post', padding='post')
    scores = model.predict(padded)
    max_index = scores[0].argmax()
    lbl = label_encoder.classes_[max_index]
    return lbl, scores[0][max_index]

In [ ]:
# sample perdiction
print(predict('this is my sample text'))

('en', 0.83365256)


In [ ]:
# Real time data from google news
test_data = [
    ('en', 'Today rural India and its villages have declared themselves'),
    ('de', 'Es ist einer dieser Momente, bei denen man dabei gewesen sein will'),
    ('fr', 'Mais rien ne permet pour l’instant de confirmer ces propos.'),
    ('it', 'Il peso della compartecipazione dei cittadini (il ticket appunto) sarà cacolato'),
    ('es', 'Después de la evaluación y las pruebas médicas, se descubrió que tenía un')
]

In [ ]:
# predict on real time data
for actual_lang, data in test_data:
    print('-----------------')
    print(f'Data:{data}')
    print(f'Predicted:{predict(data)}, Actual:{actual_lang}')

-----------------
Data:Today rural India and its villages have declared themselves
Predicted:('en', 0.5645198), Actual:en
-----------------
Data:Es ist einer dieser Momente, bei denen man dabei gewesen sein will
Predicted:('de', 0.99880373), Actual:de
-----------------
Data:Mais rien ne permet pour l’instant de confirmer ces propos.
Predicted:('fr', 0.9159891), Actual:fr
-----------------
Data:Il peso della compartecipazione dei cittadini (il ticket appunto) sarà cacolato
Predicted:('it', 0.93701303), Actual:it
-----------------
Data:Después de la evaluación y las pruebas médicas, se descubrió que tenía un
Predicted:('es', 0.88336474), Actual:es
